In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

i:\fuse\Csa\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langsmith import Client
client = Client()

In [74]:
from typing import List, Dict, Optional, TypedDict, Annotated
from operator import add

# Define State
class State(TypedDict):
    query: str
    chat_history: List[Dict[str, str]]  # List of {"user": ..., "bot": ...}
    query_types: Optional[List[Dict]]  # Will be filled by orchestrator
    subagent_outputs: Annotated[list, add]
    final_response: Optional[str]


In [75]:
from typing import List, Optional, Literal
from pydantic import BaseModel

class Parameters(BaseModel):
    search: Optional[str] = None
    type: Optional[Literal["veg", "non-veg"]] = None
    price_min: Optional[float] = None
    price_max: Optional[float] = None
    topic: Optional[str] = None

class QueryTypeItem(BaseModel):
    type: Literal["menu", "info", "escalation", "chitchat", "ambiguous"]
    parameters: Optional[Parameters] = None
    clarification: Optional[str] = None

class OrchestratorOutput(BaseModel):
    query_types: List[QueryTypeItem]


### Orchestrator stuffs

In [92]:
ORCHESTRATOR_PROMPT = """SYSTEM:
You are a restaurant assistant agent. Your task is to analyze a user query, classify its intent, and extract menu parameters if applicable. Do not answer the query; only classify and extract.

Classify the query into one or more of these types:
- menu: User wants menu details (dish names, type, price range, etc.)
- info: User asks about restaurant details (opening hours, location, contact, etc.)
- escalation: User explicitly requests human help or clearly requires it.
- chitchat: Casual or irrelevant conversation not needing a subagent.
- ambiguous: Query is unclear or missing key details. Provide a single clarifying question.

Menu parameters (for type="menu"):
- search: dish name or keyword
- type: "veg" or "non-veg"
- price_min / price_max: numeric values, if mentioned.

Info queries:
- Extract the concerned topic (e.g. "opening hours", "address", "delivery options") into `parameters` as { "topic": "<string>" }.

Special instructions:
- A query may have multiple types.
- Only ambiguous queries have a clarifying question.
- Always populate `parameters.search` for menu intents with any descriptive text from the user's query that could help search: single words, adjectives, adjective+noun phrases, quoted phrases, situational cues (e.g., "for cold weather", "spicy", "breakfast", "kid-friendly"). Do not try to normalize or expand these — just extract the phrase(s) verbatim (trimmed). If there are multiple useful phrases, join them with a comma in `search` (e.g., "spicy, cold weather").
- But if you detect a menu intent but cannot extract at least one useful menu parameter (search, type, price_min, or price_max), like "food", "something" etc then mark that intent as "ambiguous" and provide a single concise clarifying question asking for the missing detail (for example: "Do you prefer veg or non-veg, or do you want recommendations?").
- Normalize non-vegetarian types to "non-veg".
- Return data that matches the structured schema provided by the system.
"""


In [120]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import json

orchestrator_llm = llm.with_structured_output(OrchestratorOutput)

# Orchestrator Node
def orchestrator_node(state: State):
    """Classify user query and extract menu parameters for subagents."""

    # Construct LLM messages
    messages = [
        SystemMessage(content=ORCHESTRATOR_PROMPT),
        HumanMessage(content=f"Chat history:{state["chat_history"]}"),
        HumanMessage(content=state["query"])
    ]

    # Call the LLM (replace `llm` with your LangChain/LLM client)
    parsed: OrchestratorOutput = orchestrator_llm.invoke(messages)  # Should return JSON string

    state["query_types"] = parsed.model_dump()["query_types"]

    return state


In [121]:
test_state = {
    "query": "wht kind of pizza you got maybe with chicken also are you open at 5pm",
    "chat_history": [
        {"user": "Hi", "bot": "Hello! How can I help you today?"},
        {"user": "I want to know your menu.", "bot": "Sure, what type of dishes are you interested in?"}
    ]
}
resp = orchestrator_node(test_state)
print(resp['query_types'])

[{'type': 'menu', 'parameters': {'search': 'pizza, chicken', 'type': 'non-veg', 'price_min': None, 'price_max': None, 'topic': None}, 'clarification': None}, {'type': 'info', 'parameters': {'search': None, 'type': None, 'price_min': None, 'price_max': None, 'topic': 'opening hours'}, 'clarification': None}]


In [101]:
resp

{'query': 'wht kind of pizza you got maybe with chicken also are you open at 5pm',
 'chat_history': [{'user': 'Hi', 'bot': 'Hello! How can I help you today?'},
  {'user': 'I want to know your menu.',
   'bot': 'Sure, what type of dishes are you interested in?'}],
 'query_types': [{'type': 'menu',
   'parameters': {'search': 'pizza, chicken',
    'type': 'non-veg',
    'price_min': None,
    'price_max': None,
    'topic': None},
   'clarification': None},
  {'type': 'info',
   'parameters': {'search': None,
    'type': None,
    'price_min': None,
    'price_max': None,
    'topic': 'opening hours'},
   'clarification': None}]}

In [113]:
resp['query_types'][0]['parameters']

{'search': 'pizza, chicken',
 'type': 'non-veg',
 'price_min': None,
 'price_max': None,
 'topic': None}

In [32]:
filtered_params = assign_subagents(resp)
filtered_params

[Send(node='menu_agent', arg={'search': 'pizza, chicken', 'type': 'non-veg'}),
 Send(node='info_agent', arg={'topic': 'opening hours'})]

### Menu agent

In [79]:
import os, requests

def menu_agent(state: dict) -> dict:
    """
    Retrieves restaurant menu items based on the provided criteria
    and returns in the structured format for subagent aggregation.
    """
    menu_params = state.get("params", {})
    base_url = os.getenv("BASE_URL")
    items_url = base_url + "/items"

    try:
        response = requests.get(items_url, params=menu_params, timeout=10)
        response.raise_for_status()
        menu_data = response.json()
    except requests.exceptions.RequestException as e:
        menu_data = {"error": str(e)}

    # Return wrapped in subagent_outputs for operator.add merging
    return {
        "subagent_outputs": [
            {
                "type": "menu",
                "parameters": menu_params,  # only non-None params if you want
                "output": menu_data
            }
        ]
    }


In [ ]:
output = menu_agent(resp['query_types'][0]['parameters'])
output

### Info agent

In [80]:
import os, requests

def info_agent(state:dict) -> dict:
    """
    Retrieves restaurant knowledge base information based on a user query.
    """
    info_params = state.get("params", {})
    query_str = info_params.get("topic", "")

    base_url = os.getenv("BASE_URL")
    kb_url = base_url + "/knowledge/semantic-search"
    params = {
        "search": query_str
    }

    try:
        response = requests.get(kb_url, params=params, timeout=10)
        response.raise_for_status()
        info_data= response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error calling backend: {e}")
        info_data ={"error": str(e)}
    
    return {
        "subagent_outputs": [
            {
                "type": "menu",
                "parameters": info_params,  # only non-None params if you want
                "output": info_data
            }
        ]
    }


### Synthesizer

In [81]:
SYNTHESIZER_PROMPT = """SYSTEM:
You are a restaurant assistant responsible for generating the final, user-facing response.
You are given:

1. The **user query**.
2. The **recent chat history** between the user and the bot.
3. The **outputs from subagents**, each containing:
   - type: the subagent type (menu, info, etc.)
   - parameters: the input parameters used for the subagent
   - output: the result returned by that subagent

Your task:

1. Produce a coherent, human-like response using **only the subagent outputs** and memory results if available.
2. Handle different scenarios:

   a. **Direct match**: If the subagent output fully satisfies the query, use it in your answer.
   
   b. **Partial match**: If the output partially satisfies the query:
      - Highlight what matches
      - Ask a concise clarifying question if necessary
   
   c. **Too verbose / many items** (e.g., menu lists):
      - Summarize the categories or main items
      - Ask the user for a preference to provide detailed results
   
   d. **No relevant information**: Politely inform the user that the information is unavailable.
   
3. Always ensure that:
   - Responses are concise (1–3 sentences) unless a clarifying question is needed.
   - Only **one clarifying question** is asked at a time.
   - Do **not hallucinate**; rely only on the subagent outputs or memory.
   - Combine multiple subagent outputs if the query involves more than one type (e.g., menu + info).

Output format (JSON):
{
  "final_answer": "string"        # The coherent response or clarification
}
"""

In [82]:
from langchain.schema import SystemMessage, HumanMessage
from typing import Dict

# Assuming you have a structured output model like this:
from pydantic import BaseModel

class SynthesizerOutput(BaseModel):
    final_answer: str  # Can be an answer or a clarifying question

# Wrap the LLM
synthesizer_llm = llm.with_structured_output(SynthesizerOutput)

# Synthesizer Node
def synthesizer_node(state: Dict) -> Dict:
    """
    Generate a coherent response from subagent outputs.
    """
    user_query = state["query"]
    chat_history = state.get("chat_history", [])
    subagent_outputs = state.get("subagent_outputs", [])

    # Format chat history
    chat_str = "\n".join([f"User: {c['user']}\nBot: {c['bot']}" for c in chat_history])

    # Format subagent outputs
    subagent_str = ""
    for sa in subagent_outputs:
        sa_type = sa["type"]
        sa_params = sa.get("parameters", {})
        sa_output = sa.get("output")
        subagent_str += f"\n\n---\nSubagent type: {sa_type}\nParameters: {sa_params}\nOutput: {sa_output}"

    # Construct messages using the separate prompt
    messages = [
        SystemMessage(content=SYNTHESIZER_PROMPT),
        HumanMessage(content=f"User query: {user_query}\nChat history:\n{chat_str}\nSubagent outputs:\n{subagent_str}")
    ]

    # Call the LLM
    parsed: SynthesizerOutput = synthesizer_llm.invoke(messages)
    # parsed = llm.invoke(messages)

    # Store in state
    state["final_response"] = parsed.model_dump()["final_answer"]

    return state


In [24]:
test_state = {
    "query": "wht kind of dumpling you got maybe with chicken also are you open at 5pm",
    "chat_history": [
        {"user": "Hi", "bot": "Hello! How can I help you today?"},
        {"user": "I want to know your menu.", "bot": "Sure, what type of dishes are you interested in?"}
    ],
    "subagent_outputs": [
        {
            "type": "menu",
            "parameters": {'search': 'dumpling, chicken','type': 'non-veg','price_min': None,'price_max': None,'topic': None},
            "output": [
                {"name": "Spicy Paneer Curry", "type": "veg", "price": 350},
                {"name": "Spicy Chicken Momo", "type": "non-veg", "price": 450}
            ]
        },
        {
            "type": "info",
            "parameters": {'topic': 'opening hours'},
            "output": "We are located at Lazimpat, Kathmandu. The opening hours is 6 am to 4pm."
        }
    ]
}


In [25]:
# Call the node
updated_state = synthesizer_node(test_state)

# Check the final response
print(updated_state["final_response"])


We have Spicy Chicken Momo on our menu. Regarding your query about opening hours, we are open from 6 AM to 4 PM, so we are not open at 5 PM.


In [122]:
from langgraph.types import Send

def assign_subagents(state: State):
    sends = []

    for qt in state.get("query_types", []):
        if qt["type"] == "menu":
            sends.append(Send("menu_agent", {"params": {k: v for k, v in qt["parameters"].items() if v is not None}}))
        elif qt["type"] == "info":
            sends.append(Send("info_agent", {"params": {k: v for k, v in qt["parameters"].items() if v is not None}}))
        elif qt["type"] == "escalation":
            sends.append(Send("escalation_agent", {"params": qt.get("parameters", {})}))
        elif qt["type"] == "ambiguous":
            # inject synthetic chitchat output for synthesizer, no Send required
            state.setdefault("subagent_outputs", []).append({
                "type": "chitchat",
                "parameters": {},
                "output": qt["clarification"]
            })
        elif qt["type"] == "chitchat":
            # inject synthetic chitchat output for synthesizer, no Send required
            state.setdefault("subagent_outputs", []).append({
                "type": "chitchat",
                "parameters": {},
                "output": None
            })

    # If no subagents were scheduled, explicitly tell the graph to run synthesizer next
    if not sends:
        return [Send("synthesizer", {
                "query": state.get("query"),
                "chat_history": state.get("chat_history", []),
                "subagent_outputs": state.get("subagent_outputs", []),
                "memory_results": state.get("memory_results", [])})]

    return sends


In [123]:
from langgraph.graph import StateGraph, START, END

# Build the workflow
workflow_builder = StateGraph(State)

# Add nodes
workflow_builder.add_node("orchestrator", orchestrator_node)
workflow_builder.add_node("menu_agent", menu_agent)
workflow_builder.add_node("info_agent", info_agent)
# workflow_builder.add_node("escalation_agent", escalation_agent)
workflow_builder.add_node("synthesizer", synthesizer_node)

# Start edge
workflow_builder.add_edge(START, "orchestrator")

workflow_builder.add_conditional_edges("orchestrator", assign_subagents, ["menu_agent", "info_agent"])

# Collect subagent outputs and send to synthesizer
workflow_builder.add_edge("menu_agent", "synthesizer")
workflow_builder.add_edge("info_agent", "synthesizer")
# workflow_builder.add_edge("escalation_agent", "synthesizer")

# End edge
workflow_builder.add_edge("synthesizer", END)

# Compile workflow
restaurant_workflow = workflow_builder.compile()


In [124]:
# Example query
test_state = {
    "query": "lets maybe do veg today",
    "chat_history": [
        {"user": "Hi", "bot": "Hello! How can I help you today?"},
        {"user": "do you guys have something hot", "bot": "Are you looking for a hot drink or a hot dish? Do you have any other preferences, like veg or non-veg?"},
        {"user": "no i mean spicy", "bot": "We have several spicy options! Some highlights include Spicy Honey Wings, Spicy Buff & Jalapeño Pizza, sadheko buff momo, Chicken Chilly, Chicken Lollipop, Mustang Aloo, and Paneer Chilly. We also have spicy variations of Veg, Buff, and Chicken Momos. Do you have a preference for vegetarian or non-vegetarian, or a specific type of dish you're in the mood for?"},
    ],
    "subagent_outputs": []  # Start empty
}

# Invoke the workflow
final_state = restaurant_workflow.invoke(test_state)

# Final coherent response
print(final_state["final_response"])


We have several spicy vegetarian options for you, including Veg Spring Rolls and various preparations of Veg Momos like steamed, fried, kothey, jhol, chilly, and sadheko. Would you like to know more about any of these?
